In [27]:

# -*- coding: utf-8 -*-

# -- Sheet --

import pandas as pd 
import re
import os
import string
import csv
import sys
import datetime
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict

In [28]:
tweet_data = pd.read_csv("sentimenttweets.csv")
'''
# tweet_data = pd.concat([pd.read_excel(os.path.join("twitterdata", f), index_col = 0) for f in os.listdir("twitterdata")])

tweet_data = pd.read_csv("searchTweets.csv")

tweet_data = tweet_data[["ticker", "created", "sent_score"]]

tweet_data2 = tweet_data2[["ticker", "created", "sent_score"]]

tweet_data = pd.concat([tweet_data, tweet_data2], axis = 0)

tweet_data.shape

'''
print(tweet_data.head())


returns_data = pd.read_csv("stocks.csv")

returns_data.head()

# returns_data.reset_index(inplace = True)

# returns_data["Date"] = pd.to_datetime(returns_data["Date"]).dt.strftime("%m-%d-%Y")

# returns_data.rename(columns = {"level_0": "ticker", "Date": "date"}, inplace = True)

# returns_data.set_index(["ticker", "date"], inplace = True, drop = True)

# returns_data = returns_data[["Volume", "Adj Close"]]

# returns = returns_data["Adj Close"].groupby(level = 0).apply(lambda x: x / x[1:4].max())
# vol = returns_data.groupby(level = 0)["Volume"].apply(lambda x: x / x[1:4].max())

# returns_data = pd.concat([returns, vol], axis = 1)
# returns_data.rename(columns = {"Adj Close": "return", "Volume": "vol"}, inplace = True)
# returns_data.dropna(inplace = True)

# tweet_data.rename(columns = {"created": "date"}, inplace = True)
# tweet_data.set_index(["ticker", "date"], inplace = True, drop = True)

# # Drop any tweets with 0 sentiment (irrelevant/couldn't be parsed/etc.)

# tweet_data = tweet_data[tweet_data["sent_score"] != 0]

# # Calculate average sentiment score by date

# tweet_sent_data = tweet_data.groupby(["ticker", "date"])["sent_score"].agg(['mean', 'count'])
# tweet_sent_data.columns = ["sent_score", "tweet_volume"]
# tweet_sent_data["tweet_volume"] = tweet_sent_data.groupby(level = 0)["tweet_volume"].apply(lambda x: x / x[1:4].max())

# combined_data = returns_data.merge(tweet_sent_data, left_index = True, right_index = True, how = "left")

# # In case of companies that didn't have any Twitter data for a particular date... 
# combined_data.fillna(0, inplace = True)

# dt = combined_data.reset_index()

# dt["date"] = pd.to_datetime(dt["date"])
# dt.set_index(["ticker", "date"], inplace = True, drop = True)
# dt.sort_index(level = 1, ascending = False, inplace = True)

# target = dt[dt.index.get_level_values(1) == '11-27-2020']
# features = dt[dt.index.get_level_values(1) != '11-27-2020']

# target = target["return"]

  ticker                         created            id  polarity1 sentiment  \
0    MMM  Mon Apr 04 18:53:44 +0000 2022  1.510000e+18   0.106667  positive   
1    MMM  Mon Apr 04 16:55:27 +0000 2022  1.510000e+18   0.000000   neutral   
2    MMM  Mon Apr 04 16:00:46 +0000 2022  1.510000e+18   0.000000   neutral   
3    MMM  Mon Apr 04 14:30:00 +0000 2022  1.510000e+18   0.000000   neutral   
4    MMM  Fri Apr 01 14:48:13 +0000 2022  1.510000e+18   1.000000  positive   

   subjectivity1  polarity2  subjectivity2  
0       0.526667   0.106667       0.526667  
1       0.000000   0.000000       0.000000  
2       0.100000   0.000000       0.000000  
3       0.000000   0.000000       0.000000  
4       0.300000   1.000000       0.300000  


,ticker,change
0,MMM,-0.016707
1,AOS,-0.012981
2,ABT,-0.007476
3,ABBV,-0.001954
4,ABMD,-0.017808


In [29]:
tweet_agg = tweet_data.groupby(['ticker'],as_index=False).mean()
pd_data = tweet_agg.merge(returns_data)
pd_data

,ticker,id,polarity1,subjectivity1,polarity2,subjectivity2,change
0,A,1.510000e+18,0.137614,0.277176,0.132364,0.272676,-0.016241
1,AAL,1.510000e+18,0.092037,0.243148,0.055926,0.273704,-0.011634
2,AAP,1.510000e+18,0.106667,0.526667,0.106667,0.526667,-0.000425
3,AAPL,1.510000e+18,0.115024,0.333631,0.110274,0.360131,-0.015244
4,ABBV,1.510000e+18,0.287140,0.296502,0.287140,0.285391,-0.001954
...,...,...,...,...,...,...,...
369,YUM,1.510000e+18,0.000000,0.000000,0.000000,0.000000,0.000759
370,ZBH,1.510000e+18,0.000000,0.000000,0.000000,0.000000,-0.000391
371,ZBRA,1.510000e+18,0.000000,0.000000,0.000000,0.000000,0.022238
372,ZION,1.510000e+18,0.000000,0.000000,0.000000,0.000000,-0.028829


In [30]:
def get_sent(row):
    if row[2] > 0:
        val = 'positive'
    elif row[2] < 0:
        val = 'negative'
    else:
        val = 'neutral'
    return val

In [31]:
pd_data['sentiment'] = pd_data.apply(get_sent, axis=1)
pd_data.groupby(['sentiment']).mean()

,id,polarity1,subjectivity1,polarity2,subjectivity2,change
sentiment,,,,,,
negative,1.510000e+18,-0.07662,0.219143,-0.076697,0.218786,-0.019299
neutral,1.510000e+18,0.00000,0.057030,0.000000,0.057030,-0.011976
positive,1.510000e+18,0.17300,0.308270,0.164174,0.309364,-0.015691


In [32]:
target = pd_data['change']
features = pd_data[['polarity','subjectivity']]

KeyError: "None of [Index(['polarity', 'subjectivity'], dtype='object')] are in the [columns]"

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state = 0, test_size = 0.2)

In [ ]:
model = LinearRegression()
model = model.fit(X_train, y_train)

prediction = model.predict(X_test)

mse = mean_squared_error(y_test, prediction)
r2 = r2_score(y_test, prediction)

print("\nThe model performance for training set")
print("--------------------------------------")
print('MSE is {}'.format(mse))
print('R2 is {}'.format(r2))

print("\n")

In [ ]:
# KNN Regression
knn = KNeighborsRegressor()
neighbors = list(range(1,51))

param_grid = dict(n_neighbors = neighbors)
knn_grid = GridSearchCV(knn, param_grid, cv=10)

knn_grid.fit(features, target)
y_pred = list(knn_grid.predict(features))
knn_grid.score(features, target) 
knn_grid.best_params_

In [ ]:
# Graphing the MSE for each hyperparameter
error = []
for k in range(1,51):
    knn = KNeighborsRegressor(n_neighbors=k)
    y_pred = cross_val_predict(knn, features, target, cv=5)
    error.append(mean_squared_error(target,y_pred))
plt.plot(range(1,51),error)
plt.xlabel('K nearest neighbors')
plt.ylabel('Mean Squared Error')

In [ ]:


import random

random.seed(0)

train_tickers = list(random.sample(list(dt.index.get_level_values(0).unique()), 400))
test_tickers = [x for x in dt.index.get_level_values(0).unique() if x not in train_tickers]

import numpy as np 

X_train = np.array([features[features.index.get_level_values(0) == ticker].values.tolist() for ticker in train_tickers])
y_train = np.array([target[target.index.get_level_values(0) == ticker].values.tolist() for ticker in train_tickers])

X_test = np.array([features[features.index.get_level_values(0) == ticker].values.tolist() for ticker in test_tickers])
y_test = np.array([target[target.index.get_level_values(0) == ticker].values.tolist() for ticker in test_tickers])

X_test[1]




In [ ]:

# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Dropout, LSTM
# from tensorflow.keras.callbacks import EarlyStopping

# model = Sequential()
# model.add(LSTM(units = 10, return_sequences = True, input_shape = (X_train.shape[1], X_train.shape[2])))
# model.add(LSTM(units = 5))
# model.add(Dense(units = 1))

# early_stop = EarlyStopping(monitor='loss', patience=10, verbose=1)

# model.compile(optimizer = "adam", loss = "mean_squared_error")

# model.fit(X_train, y_train, epochs = 50, batch_size = 30, callbacks=[early_stop])

# model.summary()

# y_test_pred = model.predict(X_test)

# y_test_pred


In [ ]:


# import matplotlib.pyplot as plt

# plt.scatter(y_test, y_test_pred)
# plt.show()

# from sklearn.metrics import mean_squared_error

# np.sqrt(mean_squared_error(y_test, y_test_pred))

# preds = pd.DataFrame({"Predicted": y_test_pred.flatten(), "Actual": y_test.flatten()})
# preds.plot()

# results = pd.DataFrame({"Predicted": y_test_pred.flatten(), "Actual": y_test.flatten()}, index = test_tickers)

# results.head()

# t_0 = features[features.index.get_level_values(1) == '11/25/2020'][["return", "sent_score"]].reset_index(level = 1)

# res = results.merge(t_0, how = "left", left_index = True, right_index = True)

# res.rename(columns = {"return": "t-1"}, inplace = True)
# res.drop(columns = ["date"], inplace = True)

# res.head()

# ups = res[res["Predicted"] > res["t-1"]]

# downs = res[res["Predicted"] < res["t-1"]]

In [ ]:

# print("Ups: {} Downs: {}".format(ups.shape[0], downs.shape[0]))

# print("Out of {} predicted ups, {} actually went up".format(ups.shape[0], res[(res["Predicted"] > res["t-1"]) & (res["Actual"] > res["t-1"])].shape[0]))

# print("Out of {} predicted downs, {} actually went down".format(downs.shape[0], res[(res["Predicted"] < res["t-1"]) & (res["Actual"] < res["t-1"])].shape[0]))

# abs_up_ret = (ups["Actual"] - ups["t-1"]).sum()
# print("If we bought equally-weighted shares of each predicted up, we would have made: {:.2f}".format(abs_up_ret))
# print("Of a total investment of {:.2f}, that's a 1-day return of {:.2f}%".format(ups["t-1"].sum(), abs_up_ret / ups["t-1"].sum() * 100))
# print("Annualized, that figure is {:.2f}%".format((np.power(1 + abs_up_ret / ups["t-1"].sum(), 251) - 1) * 100))

# abs_down_ret = (downs["t-1"] - downs["Actual"]).sum()
# print("If we shorted equally-weighted shares of each predicted down, we would have made: {:.2f}".format(abs_down_ret))
# print("Of a total investment of {:.2f} (@150% margin), that's a 1-day return of {:.2f}%".format(downs["t-1"].sum() * 1.5, abs_down_ret / (downs["t-1"].sum() * 1.5) * 100))
# print("Annualized, that figure is {:.2f}%".format((np.power(1 + abs_down_ret / (downs["t-1"].sum() * 1.5), 251) - 1) * 100))

# # Impact of sentiment on performance

# top_sent = res.nlargest(20, columns = "sent_score")
# low_sent = res.nsmallest(20, columns = "sent_score")

# print("Of 20 companies with highest sentiment at t-1, {} went up, {} went down".format(top_sent[top_sent["Actual"] > top_sent["t-1"]].shape[0], top_sent[top_sent["Actual"] < top_sent["t-1"]].shape[0]))

# print("Equal-weighted price return (long highest sentiment companies): {:.2f}".format((top_sent["Actual"] - top_sent["t-1"]).sum()))

# print("Of 20 companies with lowest sentiment at t-1, {} went up, {} went down".format(low_sent[low_sent["Actual"] > low_sent["t-1"]].shape[0], low_sent[low_sent["Actual"] < low_sent["t-1"]].shape[0]))

# print("Equal-weighted price return (short lowest sentiment companies): {:.2f}".format((low_sent["t-1"] - low_sent["Actual"]).sum()))

# tr = (top_sent["Actual"] - top_sent["t-1"]).sum() + (low_sent["t-1"] - low_sent["Actual"]).sum()
# print("Strategy: Long Top 20 Sentiment, Short Worst 20 Sentiment")
# print("Net price return: {:.2f}".format(tr))
# print("Net 1-day : {:.2f}%\nAnnualized: {:.2f}%".format(100 * tr / (top_sent["t-1"].sum() + 1.5 * low_sent["t-1"].sum()), (np.power(1 + tr / (top_sent["t-1"].sum() + 1.5 * low_sent["t-1"].sum()), 251) - 1) * 100))
